# GEM-PRO - Calculating Protein Properties
This notebook gives an example of how to **calculate protein properties** for a list of proteins, by first pulling information from UniProt and then using the 3D structure files to calculate the desired properties

<div class="alert alert-info">

**Input:** List of gene IDs

</div>

<div class="alert alert-info">

**Output:** Representative protein structures and properties associated with them

</div>

## Imports

In [1]:
import sys
import logging

In [2]:
# Import the GEM-PRO class
from ssbio.pipeline.gempro import GEMPRO

In [3]:
# Printing multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Logging

Set the logging level in `logger.setLevel(logging.<LEVEL_HERE>)` to specify how verbose you want the pipeline to be. Debug is most verbose.

- `CRITICAL`
     - Only really important messages shown
- `ERROR`
     - Major errors
- `WARNING`
     - Warnings that don't affect running of the pipeline
- `INFO` (default)
     - Info such as the number of structures mapped per gene
- `DEBUG`
     - Really detailed information that will print out a lot of stuff
     
<div class="alert alert-warning">

**Warning:** 
`DEBUG` mode prints out a large amount of information, especially if you have a lot of genes. This may stall your notebook!
</div>

In [4]:
# Create logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # SET YOUR LOGGING LEVEL HERE #

In [5]:
# Other logger stuff for Jupyter notebooks
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] %(levelname)s: %(message)s', datefmt="%Y-%m-%d %H:%M")
handler.setFormatter(formatter)
logger.handlers = [handler]

## Initialization of the project

Set these three things:

- `ROOT_DIR`
    - The directory where a folder named after your `PROJECT` will be created
- `PROJECT`
    - Your project name
- `LIST_OF_GENES`
    - Your list of gene IDs
    
A directory will be created in `ROOT_DIR` with your `PROJECT` name. The folders are organized like so:
```
    ROOT_DIR
    └── PROJECT
        ├── data  # General storage for pipeline outputs
        ├── model  # SBML and GEM-PRO models are stored here
        ├── genes  # Per gene information
        │   ├── <gene_id1>  # Specific gene directory
        │   │   └── protein
        │   │       ├── sequences  # Protein sequence files, alignments, etc.
        │   │       └── structures  # Protein structure files, calculations, etc.
        │   └── <gene_id2>
        │       └── protein
        │           ├── sequences
        │           └── structures
        ├── reactions  # Per reaction information
        │   └── <reaction_id1>  # Specific reaction directory
        │       └── complex
        │           └── structures  # Protein complex files
        └── metabolites  # Per metabolite information
            └── <metabolite_id1>  # Specific metabolite directory
                └── chemical
                    └── structures  # Metabolite 2D and 3D structure files
                
```

<div class="alert alert-info">**Note:** Methods for protein complexes and metabolites are still in development.</div>

In [6]:
# SET FOLDERS AND DATA HERE
import tempfile
ROOT_DIR = tempfile.gettempdir()

PROJECT = 'ssbio_protein_properties'
LIST_OF_GENES = ['b1276', 'b0118']

In [7]:
# Create the GEM-PRO project
my_gempro = GEMPRO(gem_name=PROJECT, root_dir=ROOT_DIR, genes_list=LIST_OF_GENES, pdb_file_type='pdb')

[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: Creating GEM-PRO project directory in folder /tmp
[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: /tmp/ssbio_protein_properties: GEM-PRO project location
[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: 2: number of genes


## Mapping gene ID --> sequence

First, we need to map these IDs to their protein sequences. There are 2 ID mapping services provided to do this - through **KEGG** or **UniProt**. The end goal is to map a UniProt ID to each ID, since there is a comprehensive mapping (and some useful APIs) between UniProt and the PDB.

<p><div class="alert alert-info">**Note:** You only need to map gene IDs using one service. However you can run both if some genes don't map in one service and do map in another!</div></p>

### Methods

In [8]:
# UniProt mapping
my_gempro.uniprot_mapping_and_metadata(model_gene_source='ENSEMBLGENOME_ID')
print('Missing UniProt mapping: ', my_gempro.missing_uniprot_mapping)
my_gempro.df_uniprot_metadata.head()

[2017-09-24 05:22] [root] INFO: getUserAgent: Begin
[2017-09-24 05:22] [root] INFO: getUserAgent: user_agent: EBI-Sample-Client/ (services.py; Python 3.5.3; Linux) Python-requests/2.14.2
[2017-09-24 05:22] [root] INFO: getUserAgent: End


[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: 2/2: number of genes mapped to UniProt
[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: Completed ID mapping --> UniProt. See the "df_uniprot_metadata" attribute for a summary dataframe.



Missing UniProt mapping:  []


,uniprot,reviewed,gene_name,kegg,refseq,pdbs,pfam,description,entry_date,entry_version,seq_date,seq_version,sequence_file,metadata_file
gene,,,,,,,,,,,,,,
b0118,P36683,False,acnB,ecj:JW0114;eco:b0118,NP_414660.1;WP_001307570.1,1L5J,PF00330;PF06434;PF11791,Aconitate hydratase B,2017-08-30,162,1997-11-01,3,P36683.fasta,P36683.xml
b1276,P25516,False,acnA,ecj:JW1268;eco:b1276,NP_415792.1;WP_000099535.1,NaN,PF00330;PF00694,Aconitate hydratase A,2017-08-30,150,2008-01-15,3,P25516.fasta,P25516.xml


In [9]:
# Set representative sequences
my_gempro.set_representative_sequence()
print('Missing a representative sequence: ', my_gempro.missing_representative_sequence)
my_gempro.df_representative_sequences.head()

[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: 2/2: number of genes with a representative sequence
[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: See the "df_representative_sequences" attribute for a summary dataframe.



Missing a representative sequence:  []


,uniprot,kegg,pdbs,sequence_file,metadata_file
gene,,,,,
b0118,P36683,ecj:JW0114;eco:b0118,1L5J,P36683.fasta,P36683.xml
b1276,P25516,ecj:JW1268;eco:b1276,NaN,P25516.fasta,P25516.xml


## Mapping representative sequence --> structure

These are the ways to map sequence to structure:

1. Use the UniProt ID and their automatic mappings to the PDB
2. BLAST the sequence to the PDB
3. Make homology models or 
4. Map to existing homology models

You can only utilize option #1 to map to PDBs if there is a mapped UniProt ID set in the representative sequence. If not, you'll have to BLAST your sequence to the PDB or make a homology model. You can also run both for maximum coverage.

### Methods

In [10]:
# Mapping using the PDBe best_structures service
my_gempro.map_uniprot_to_pdb(seq_ident_cutoff=.3)
my_gempro.df_pdb_ranking.head()

[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: Mapping UniProt IDs --> PDB IDs...
[2017-09-24 05:22] [root] INFO: getUserAgent: Begin
[2017-09-24 05:22] [root] INFO: getUserAgent: user_agent: EBI-Sample-Client/ (services.py; Python 3.5.3; Linux) Python-requests/2.14.2
[2017-09-24 05:22] [root] INFO: getUserAgent: End


[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: 1/2: number of genes with at least one experimental structure
[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: Completed UniProt --> best PDB mapping. See the "df_pdb_ranking" attribute for a summary dataframe.


,pdb_id,pdb_chain_id,uniprot,experimental_method,resolution,coverage,start,end,unp_start,unp_end,rank
gene,,,,,,,,,,,
b0118,1l5j,A,P36683,X-ray diffraction,2.4,1,1,865,1,865,1
b0118,1l5j,B,P36683,X-ray diffraction,2.4,1,1,865,1,865,2


In [11]:
# Mapping using BLAST
my_gempro.blast_seqs_to_pdb(all_genes=True, seq_ident_cutoff=.7, evalue=0.00001)
my_gempro.df_pdb_blast.head(2)

[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: Completed sequence --> PDB BLAST. See the "df_pdb_blast" attribute for a summary dataframe.
[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: 0: number of genes with additional structures added from BLAST
[2017-09-24 05:22] [ssbio.pipeline.gempro] WARNING: Empty dataframe


""


In [12]:
import pandas as pd
import os.path as op

In [13]:
# Creating manual mapping dictionary for ECOLI I-TASSER models
homology_models = '/home/nathan/projects_archive/homology_models/ECOLI/zhang/'
homology_models_df = pd.read_csv('/home/nathan/projects_archive/homology_models/ECOLI/zhang_data/160804-ZHANG_INFO.csv')
tmp = homology_models_df[['zhang_id','model_file','m_gene']].drop_duplicates()
tmp = tmp[pd.notnull(tmp.m_gene)]

homology_model_dict = {}

for i,r in tmp.iterrows():
    homology_model_dict[r['m_gene']] = {r['zhang_id']: {'model_file':op.join(homology_models, r['model_file']),
                                                        'file_type':'pdb'}}
    
my_gempro.get_manual_homology_models(homology_model_dict)

[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: Updated homology model information for 2 genes.


In [14]:
# Creating manual mapping dictionary for ECOLI SUNPRO models
homology_models = '/home/nathan/projects_archive/homology_models/ECOLI/sunpro/'
homology_models_df = pd.read_csv('/home/nathan/projects_archive/homology_models/ECOLI/sunpro_data/160609-SUNPRO_INFO.csv')
tmp = homology_models_df[['sunpro_id','model_file','m_gene']].drop_duplicates()
tmp = tmp[pd.notnull(tmp.m_gene)]

homology_model_dict = {}

for i,r in tmp.iterrows():
    homology_model_dict[r['m_gene']] = {r['sunpro_id']: {'model_file':op.join(homology_models, r['model_file']),
                                                         'file_type':'pdb'}}
    
my_gempro.get_manual_homology_models(homology_model_dict)

[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: Updated homology model information for 2 genes.


## Downloading and ranking structures

### Methods

<div class="alert alert-warning">

**Warning:** 
Downloading all PDBs takes a while, since they are also parsed for metadata. You can skip this step and just set representative structures below if you want to minimize the number of PDBs downloaded.

</div>

In [15]:
# Download all mapped PDBs and gather the metadata
my_gempro.pdb_downloader_and_metadata()
my_gempro.df_pdb_metadata.head(2)

[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: Updated PDB metadata dataframe. See the "df_pdb_metadata" attribute for a summary dataframe.
[2017-09-24 05:22] [ssbio.pipeline.gempro] INFO: Saved 1 structures total


,pdb_id,pdb_title,description,experimental_method,mapped_chains,resolution,chemicals,taxonomy_name,structure_file
gene,,,,,,,,,
b0118,1l5j,CRYSTAL STRUCTURE OF E. COLI ACONITASE B.,Aconitate hydratase 2 (E.C.4.2.1.3),X-ray diffraction,A;B,2.4,F3S;TRA,Escherichia coli,1l5j.pdb


In [16]:
# Set representative structures
my_gempro.set_representative_structure()
my_gempro.df_representative_structures.head()

[2017-09-24 05:23] [ssbio.pipeline.gempro] INFO: 2/2: number of genes with a representative structure
[2017-09-24 05:23] [ssbio.pipeline.gempro] INFO: See the "df_representative_structures" attribute for a summary dataframe.


,id,is_experimental,file_type,structure_file
gene,,,,
b0118,REP-1l5j,True,pdb,1l5j-A_clean.pdb
b1276,REP-ACON1_ECOLI,False,pdb,ACON1_ECOLI_model1_clean-X_clean.pdb


### Computing sequence and structure properties

In [17]:
# Requires EMBOSS "pepstats" program
# See the ssbio wiki for more information: https://github.com/SBRG/ssbio/wiki/Software-Installations
# Install using:
# sudo apt-get install emboss
my_gempro.get_sequence_properties()

In [18]:
# Requires SCRATCH installation, replace path_to_scratch with own path to script
# See the ssbio wiki for more information: https://github.com/SBRG/ssbio/wiki/Software-Installations
my_gempro.get_scratch_predictions(path_to_scratch='/home/nathan/software/SCRATCH-1D_1.1/bin/run_SCRATCH-1D_predictors.sh', 
                                  results_dir=my_gempro.data_dir,
                                  num_cores=2)

[2017-09-24 05:23] [ssbio.pipeline.gempro] INFO: /tmp/ssbio_protein_properties/data/ssbio_protein_properties_cds.faa: wrote all representative sequences to file


[2017-09-24 05:23] [ssbio.pipeline.gempro] INFO: 2/2: number of genes with SCRATCH predictions loaded


In [19]:
my_gempro.get_disulfide_bridges(representatives_only=False)

In [20]:
# Requires DSSP installation
# See the ssbio wiki for more information: https://github.com/SBRG/ssbio/wiki/Software-Installations
my_gempro.get_dssp_annotations(representatives_only=False)

In [21]:
# Requires MSMS installation
# See the ssbio wiki for more information: https://github.com/SBRG/ssbio/wiki/Software-Installations
my_gempro.get_msms_annotations(representatives_only=False)

---------------

### Extracting residue-level properties for a specific protein feature

##### Looking at metal binding sites

In [24]:
metal_info = []

for g in my_gempro.genes:
    for f in g.protein.representative_sequence.features:
        if 'metal' in f.type.lower():
            res_info = g.protein.get_residue_annotations(f.location.end, use_representatives=True)
            res_info['gene_id'] = g.id
            res_info['seq_id'] = g.protein.representative_sequence.id
            res_info['struct_id'] = g.protein.representative_structure.id
            res_info['chain_id'] = g.protein.representative_chain
            metal_info.append(res_info)
        
cols = ['gene_id', 'seq_id', 'struct_id', 'chain_id',
        'seq_residue', 'seq_resnum', 'struct_residue','struct_resnum',
        'seq_SS-sspro','seq_SS-sspro8','seq_RSA-accpro','seq_RSA-accpro20',
        'struct_SS-dssp','struct_RSA-dssp', 'struct_ASA-dssp', 
        'struct_PHI-dssp', 'struct_PSI-dssp', 'struct_CA_DEPTH-msms', 'struct_RES_DEPTH-msms']

pd.DataFrame.from_records(metal_info, columns=cols).set_index(['gene_id', 'seq_id', 'struct_id', 'chain_id', 'seq_resnum'])

seq_residue struct_residue  \
gene_id seq_id struct_id       chain_id seq_resnum                              
b1276   P25516 REP-ACON1_ECOLI X        435                  C              C   
                                        501                  C              C   
                                        504                  C              C   
b0118   P36683 REP-1l5j        A        710                  C              C   
                                        769                  C              C   
                                        772                  C              C   

                                                    struct_resnum  \
gene_id seq_id struct_id       chain_id seq_resnum                  
b1276   P25516 REP-ACON1_ECOLI X        435                   435   
                                        501                   501   
                                        504                   504   
b0118   P36683 REP-1l5j        A        710                   710   
                                        769                   769   
                                        772                   772   

                                                   seq_SS-sspro seq_SS-sspro8  \
gene_id seq_id struct_id       chain_id seq_resnum                              
b1276   P25516 REP-ACON1_ECOLI X        435                   H             H   
                                        501                   C             C   
                                        504                   H             G   
b0118   P36683 REP-1l5j        A        710                   C             T   
                                        769                   C             C   
                                        772                   H             G   

                                                   seq_RSA-accpro  \
gene_id seq_id struct_id       chain_id seq_resnum                  
b1276   P25516 REP-ACON1_ECOLI X        435                     -   
                                        501                     -   
                                        504                     -   
b0118   P36683 REP-1l5j        A        710                     -   
                                        769                     -   
                                        772                     -   

                                                    seq_RSA-accpro20  \
gene_id seq_id struct_id       chain_id seq_resnum                     
b1276   P25516 REP-ACON1_ECOLI X        435                       10   
                                        501                        0   
                                        504                        0   
b0118   P36683 REP-1l5j        A        710                       10   
                                        769                        5   
                                        772                        5   

                                                   struct_SS-dssp  \
gene_id seq_id struct_id       chain_id seq_resnum                  
b1276   P25516 REP-ACON1_ECOLI X        435                     H   
                                        501                     S   
                                        504                     G   
b0118   P36683 REP-1l5j        A        710                     T   
                                        769                     -   
                                        772                     G   

                                                    struct_RSA-dssp  \
gene_id seq_id struct_id       chain_id seq_resnum                    
b1276   P25516 REP-ACON1_ECOLI X        435                0.059259   
                                        501                0.088889   
                                        504                0.259259   
b0118   P36683 REP-1l5j        A        710                0.118519   
                                        769                0.088889   
                    

### Visualizing residues

In [28]:
for g in my_gempro.genes:
    
    # Gather residue numbers
    metal_binding_structure_residues = []
    for f in g.protein.representative_sequence.features:
        if 'metal' in f.type.lower():
            res_info = g.protein.get_residue_annotations(f.location.end, use_representatives=True)
            metal_binding_structure_residues.append(res_info['struct_resnum'])
    print(metal_binding_structure_residues)
    
    # Display structure
    g.protein.representative_structure.view_structure_and_highlight_residues(structure_resnums=metal_binding_structure_residues)

[2017-09-24 05:34] [ssbio.protein.structure.structprop] INFO: Selection: ( :X ) and not hydrogen and ( 504 or 435 or 501 )


[435, 501, 504]


[2017-09-24 05:34] [ssbio.protein.structure.structprop] INFO: Selection: ( :A ) and not hydrogen and ( 769 or 772 or 710 )


[710, 769, 772]


### Comparing features in different structures of the same protein

In [31]:
# Run all sequence to structure alignments
for g in my_gempro.genes:
    for s in g.protein.structures:
        g.protein.align_seqprop_to_structprop(seqprop=g.protein.representative_sequence, structprop=s)

In [35]:
metal_info_compared = []

for g in my_gempro.genes:
    for f in g.protein.representative_sequence.features:
        if 'metal' in f.type.lower():
            for s in g.protein.structures:
                for c in s.mapped_chains:
                    res_info = g.protein.get_residue_annotations(seq_resnum=f.location.end, 
                                                                 seqprop=g.protein.representative_sequence,
                                                                 structprop=s, chain_id=c,
                                                                 use_representatives=False)
                    res_info['gene_id'] = g.id
                    res_info['seq_id'] = g.protein.representative_sequence.id
                    res_info['struct_id'] = s.id
                    res_info['chain_id'] = c
                    metal_info_compared.append(res_info)
    
cols = ['gene_id', 'seq_id', 'struct_id', 'chain_id',
        'seq_residue', 'seq_resnum', 'struct_residue','struct_resnum',
        'seq_SS-sspro','seq_SS-sspro8','seq_RSA-accpro','seq_RSA-accpro20',
        'struct_SS-dssp','struct_RSA-dssp', 'struct_ASA-dssp', 
        'struct_PHI-dssp', 'struct_PSI-dssp', 'struct_CA_DEPTH-msms', 'struct_RES_DEPTH-msms']

pd.DataFrame.from_records(metal_info_compared, columns=cols).sort_values(by=['seq_resnum','struct_id','chain_id']).set_index(['gene_id','seq_id','seq_resnum','seq_residue','struct_id'])

chain_id struct_residue  \
gene_id seq_id seq_resnum seq_residue struct_id                                 
b1276   P25516 435        C           ACON1_ECOLI            X              C   
                                      E01201                 X              C   
                                      REP-ACON1_ECOLI        X              C   
               501        C           ACON1_ECOLI            X              C   
                                      E01201                 X              C   
                                      REP-ACON1_ECOLI        X              C   
               504        C           ACON1_ECOLI            X              C   
                                      E01201                 X              C   
                                      REP-ACON1_ECOLI        X              C   
b0118   P36683 710        C           1l5j                   A              C   
                                      1l5j                   B              C   
                                      ACON2_ECOLI            X              C   
                                      E00113                 X              C   
                                      REP-1l5j               A              C   
               769        C           1l5j                   A              C   
                                      1l5j                   B              C   
                                      ACON2_ECOLI            X              C   
                                      E00113                 X              C   
                                      REP-1l5j               A              C   
               772        C           1l5j                   A              C   
                                      1l5j                   B              C   
                                      ACON2_ECOLI            X              C   
                                      E00113                 X              C   
                                      REP-1l5j               A              C   

                                                       struct_resnum  \
gene_id seq_id seq_resnum seq_residue struct_id                        
b1276   P25516 435        C           ACON1_ECOLI                435   
                                      E01201                     435   
                                      REP-ACON1_ECOLI            435   
               501        C           ACON1_ECOLI                501   
                                      E01201                     501   
                                      REP-ACON1_ECOLI            501   
               504        C           ACON1_ECOLI                504   
                                      E01201                     504   
                                      REP-ACON1_ECOLI            504   
b0118   P36683 710        C           1l5j                       710   
                                      1l5j                       710   
                                      ACON2_ECOLI                710   
                                      E00113                     710   
                                      REP-1l5j                   710   
               769        C           1l5j                       769   
                                      1l5j                       769   
                                      ACON2_ECOLI                769   
                                      E00113                     769   
                                      REP-1l5j                   769   
               772        C           1l5j                       772   
                                      1l5j                       772   
                                      ACON2_ECOLI                772   
                                      E00113                     772   
                                      REP-1l5j                   772   

                                                      seq_SS-sspro  \
gene